In [397]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
import requests

In [398]:
!ls

 adrian_food_global_pricing.ipynb     'adrian pre-modeling.ipynb'
 adrian_food_global_pricing_v2.ipynb  'Allan_Adding CPI to dataset.ipynb'
 adrian-new-modelling.ipynb	      'Allan_Cleaned Food Prices Grouped.ipynb'
 adrian_nutrients.ipynb		      'Allan_Cleaned Food Prices.ipynb'
 adrian-nz-types-prices.ipynb	      'Allan_Data_ Exploration.ipynb'


## API food

In [767]:
def nutrients_redux(food):
    
    product=food
   
    
    API_KEY = 'zYKJIFsOxODivrK9dGg8Y2UUWxkr2j2HMNYXLLwf'
    
    params = {
        'api_key': API_KEY,
        'query': product,
        'pageSize': 1,
        'numberOfResultsPerPage': 1,
        #'dataType':"Foundation"
    }
    response = requests.get(' https://api.nal.usda.gov/fdc/v1/foods/search', params=params).json()
    
    protein = 'not found'
    fat = 'not found'
    carb = 'not found'
    sugar = 'not found'
    sodium = 'not found'
    cholesterol = 'not found'
    calcium = 'not found'
    energy = 'not found'
    size = 100
    unit = 'g'
    category='not found'
    try:
        
        for element in response['foods'][0]['foodNutrients']:
            if element['nutrientName'] == 'Protein':
                protein = element['value']
            if element['nutrientName'] == 'Total lipid (fat)':
                fat = element['value']
            if element['nutrientName'] == 'Carbohydrate, by difference':
                carb = element['value']
            if element['nutrientName'] == 'Sugars, Total NLEA' or 'Sugars, total including NLEA':
                sugar = element['value']
            if element['nutrientName'] == 'Sodium, Na':
                sodium = element['value']
            if element['nutrientName'] == 'Cholesterol':
                cholesterol = element['value']
            if element['nutrientName'] == 'Calcium, Ca':
                calcium = element['value']
            if element['nutrientName'] == 'Energy':
                energy = element['value']
        size = response['foods'][0]['servingSize']
        unit = response['foods'][0]['servingSizeUnit']
    except:
        print(f'missing branded info for {food}')
    
    result = {
        'protein': protein,
        'fat': fat,
        'carb': carb,
        'sugar': sugar,
        'sodium': sodium,
        'cholesterol': cholesterol,
        'calcium': calcium,
        'kcal': energy,
        'size': size,
        'unit': unit,
        'category': response['foods'][0]['foodCategory']
    }
    return result

In [766]:
def nutrients_redux_2(food):
    
    product=food
   
    
    API_KEY = 'zYKJIFsOxODivrK9dGg8Y2UUWxkr2j2HMNYXLLwf'
    
    params = {
        'api_key': API_KEY,
        'query': product,
        'pageSize': 1,
        'numberOfResultsPerPage': 1,
        'dataType': "Foundation"
    }
    response = requests.get(' https://api.nal.usda.gov/fdc/v1/foods/search', params=params).json()
    
    protein = 'not found'
    fat = 'not found'
    carb = 'not found'
    sugar = 'not found'
    sodium = 'not found'
    cholesterol = 'not found'
    calcium = 'not found'
    energy = 'not found'
    size = 100
    unit = 'g'
    category = 'not found'
    try:
        
        for element in response['foods'][0]['foodNutrients']:
            if element['nutrientName'] == 'Protein':
                protein = element['value']
            if element['nutrientName'] == 'Total lipid (fat)':
                fat = element['value']
            if element['nutrientName'] == 'Carbohydrate, by difference':
                carb = element['value']
            if element['nutrientName'] == 'Sugars, Total NLEA' or 'Sugars, total including NLEA':
                sugar = element['value']
            if element['nutrientName'] == 'Sodium, Na':
                sodium = element['value']
            if element['nutrientName'] == 'Cholesterol':
                cholesterol = element['value']
            if element['nutrientName'] == 'Calcium, Ca':
                calcium = element['value']
            if element['nutrientName'] == 'Energy':
                energy = element['value']
        size = response['foods'][0]['servingSize']
        unit = response['foods'][0]['servingSizeUnit']
    except:
        print(f'missing foundation info for {food}')
    
    result = {
        'protein': protein,
        'fat': fat,
        'carb': carb,
        'sugar': sugar,
        'sodium': sodium,
        'cholesterol': cholesterol,
        'calcium': calcium,
        'kcal': energy,
        'size': size,
        'unit': unit,
        'category': response['foods'][0]['foodCategory']
    }
    return result

In [401]:
#a function to call the other two functions if the first fails
def nutrients_super(food):
    try:
        product=food

        result = nutrients_redux_2(product)

        if sum(x== 'not found' for x in result.values()) >= 3:
            result = nutrients_redux(product)
            print('switched to branded')
        else:
            print('foundation')
    except:
        print('crashed, forced nutrients_redux')
        result = nutrients_redux(product)
    return result

In [402]:
def nutrients(food):
    
    product=food
   
    
    API_KEY = 'zYKJIFsOxODivrK9dGg8Y2UUWxkr2j2HMNYXLLwf'
    
    params = {
        'api_key': API_KEY,
        'query': product,
        'pageSize': 1,
        'numberOfResultsPerPage': 1
    }
    response = requests.get(' https://api.nal.usda.gov/fdc/v1/foods/search', params=params).json()
    
    for element in response['foods'][0]['foodNutrients']:
        print(f'{product} has {element["value"]} {element["unitName"]} of {element["nutrientName"]}')

## Getting the table NZ

In [403]:
nz_df = pd.read_csv('../raw_data/NZ_food_prices_2020.csv')
#nz_df.head()

In [404]:
nz_df['Series_title_1'].unique()

array(['Oranges', 'Bananas', 'Apples', 'Kiwifruit',
       'Sultanas (supermarket only)',
       'Peaches - canned (supermarket only)', 'Lettuce', 'Broccoli',
       'Cabbage', 'Tomatoes', 'Carrots', 'Mushrooms', 'Potatoes',
       'Peas - frozen (supermarket only)', 'Beef steak - blade',
       'Beef steak - porterhouse/sirloin', 'Beef - mince',
       'Pork - loin chops', 'Lamb - chops', 'Sausages',
       'Tuna - canned (supermarket only)', 'Biscuits - chocolate',
       'Breakfast biscuits', 'Flour - white (supermarket only)',
       'Rice - long grain', 'Milk - standard homogenised',
       'Yoghurt - flavoured', 'Cheese - mild cheddar (supermarket only)',
       'Eggs', 'Butter - salted', 'Sugar - white',
       'Chocolate - block (supermarket only)', 'Spaghetti - canned',
       'Coffee - instant', 'Tea bags (supermarket only)', 'Soft drink',
       'Bottled water', 'Fish and chips', 'Meat pie - hot',
       'Fruit juice - apple based (supermarket only)', 'Potato crisps',
      

In [405]:
nutrients_super('orange')

missing foundation info for orange
foundation


{'protein': 0.91,
 'fat': 0.15,
 'carb': 11.8,
 'sugar': 8.57,
 'sodium': 9.0,
 'cholesterol': 'not found',
 'calcium': 43.0,
 'kcal': 196,
 'size': 100,
 'unit': 'g',
 'category': 'Fruits and Fruit Juices'}

In [406]:
nz_df['type'] = nz_df['Series_title_1'].apply(lambda x: x.split('(')[0])
nz_df['type'] = nz_df['type'].apply(lambda x: x.replace('-',''))
nz_df['type'] = nz_df['type'].apply(lambda x: x.strip(' '))
nz_df['type'] = nz_df['type'].apply(lambda x: string_cleaner(x))

nz_df['type'].head(15)

0               Oranges
1               Bananas
2                Apples
3             Kiwifruit
4              Sultanas
5       Peaches  canned
6               Lettuce
7              Broccoli
8               Cabbage
9              Tomatoes
10              Carrots
11            Mushrooms
12             Potatoes
13         Peas  frozen
14    Beef steak  blade
Name: type, dtype: object

In [407]:
nz_df['Unit'].unique()

array(['1kg', '375g', '410g', '185g', '200g', '5kg', '2 litres', '0',
       '500g', '250g', '420g', '100g', '5 litres', '750ml', 'each',
       '3 litre', '150g', '560g', '700g', '600g', '110g', '6 pack',
       '125g', '5 litre', ' 50g', '300g', '900g', '750g', '1 litre',
       '130g', '800g', '425g', '340g', '210g', '600ml', '300ml', '350ml',
       '250ml', '400g', '380ml', ' 15g'], dtype=object)

In [408]:
def string_cleaner(string):
    chars = '-@!#$%&()\/:,'

    for char in chars:
        string = string.replace(char, ' ')
    return string

string_cleaner('Beef steak  porterhouse/sirloin')

'Beef steak  porterhouse sirloin'

In [409]:
#making a function to split the number and the unit 

def spliter_1(string):
    import re
    #for value in string:
    try:
        match = re.match(r"([0-9]+)([a-z]+)", string.strip(' ').replace(' ',''), re.I)
        items = match.groups()
        return items[0]
    except:
        
        return string

In [410]:
def spliter_2(string):
    import re
    #for value in string:
    try:
        match = re.match(r"([0-9]+)([a-z]+)", string.strip(' ').replace(' ',''), re.I)
        items = match.groups()
        return items[1]
    except:
        
        return string

In [411]:
print(spliter_1(nz_df['Unit'][0]), spliter_2(nz_df['Unit'][0]))

1 kg


In [412]:
nz_df['Unit'].value_counts()

1kg         47
0           22
each        10
500g        10
250g         7
200g         5
375g         4
700g         4
300g         3
6 pack       3
2 litres     3
100g         3
150g         2
750ml        2
1 litre      2
5kg          2
400g         2
300ml        1
350ml        1
600ml        1
250ml        1
210g         1
340g         1
425g         1
800g         1
380ml        1
130g         1
110g         1
750g         1
900g         1
 50g         1
5 litre      1
125g         1
600g         1
560g         1
3 litre      1
5 litres     1
420g         1
185g         1
410g         1
 15g         1
Name: Unit, dtype: int64

In [413]:
nz_df.head(1)

,Unnamed: 0,Series_reference,Period,Data_value,STATUS,UNITS,Subject,Group,Series_title_1,Series_title_2,Series_title_3,Unit,type
0,163,CPIM.SAP0100,2020.01,4.27,FINAL,Dollars,Consumers Price Index - CPI,Food Price Index Selected Monthly Weighted Ave...,Oranges,1kg,0,1kg,Oranges


In [414]:
#separating the value from the unit and creating a new column for each one
nz_df['measure'] = nz_df['Unit'].apply(lambda x: spliter_1(x))
nz_df['unit'] = nz_df['Unit'].apply(lambda x: spliter_2(x))


In [416]:
nz_df.head(1)

,Unnamed: 0,Series_reference,Period,Data_value,STATUS,UNITS,Subject,Group,Series_title_1,Series_title_2,Series_title_3,Unit,type,measure,unit
0,163,CPIM.SAP0100,2020.01,4.27,FINAL,Dollars,Consumers Price Index - CPI,Food Price Index Selected Monthly Weighted Ave...,Oranges,1kg,0,1kg,Oranges,1,kg


In [431]:
clean_df = nz_df.filter(items=['type','Data_value','measure','unit'])

In [432]:
clean_df.isnull().sum()

type          0
Data_value    0
measure       0
unit          0
dtype: int64

In [604]:
clean_df['unit'].value_counts()

kg       132
litre     22
Name: unit, dtype: int64

In [443]:
clean_df.loc[clean_df['type'] == 'Eggs'].measure = 370

/home/drifter/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [449]:
#function to manually substitute the value of measure and unit for every strange row
def weird(tipo, measure, unit):
    
    for index, rows in clean_df.loc[clean_df['type'] == tipo].iterrows():
        clean_df.loc[index, 'measure']= measure
        clean_df.loc[index, 'unit']= unit

In [569]:
weird('Flat bread  pita', 60, 'g')

In [575]:
clean_df.loc[clean_df['measure'] == 'each']

,type,Data_value,measure,unit


In [568]:
clean_df.loc[79, 'type']

'Flat bread  pita'

In [ ]:
#eliminating rows that weren't able to be changed

#clean_df.drop(90, inplace=True)

In [588]:
clean_df['measure'].unique()

array(['1', '375', '410', '185', '200', '5', '2', 1, 370, '500', '250',
       '420', '100', 250, '750', 450, 600, '3', '150', '560', '700',
       '600', '110', 3, 1.5, 200, '125', 2.6, '50', 112, '300', 60, 226,
       90, '900', 220, '130', '800', '425', '340', '210', '350', 95, 100,
       16, 57, 300, 4, '400', 500, '380', '15', 28], dtype=object)

In [591]:
clean_df['measure'] = pd.to_numeric(clean_df['measure'])

In [592]:
clean_df['measure'].dtype

dtype('float64')

In [602]:
#standarize everything to kg or litre

clean_df['measure'] = clean_df.apply(lambda x: x['measure']/1000 if x['unit']== 'g' else x['measure'], axis=1)
clean_df['unit'] = clean_df.apply(lambda x: 'kg' if x['unit']=='g' else x['unit'], axis=1)

clean_df['measure'] = clean_df.apply(lambda x: x['measure']/1000 if x['unit']== 'ml' else x['measure'], axis=1)
clean_df['unit'] = clean_df.apply(lambda x: 'litre' if x['unit']=='ml' else x['unit'], axis=1)

clean_df['unit'] = clean_df.apply(lambda x: 'litre' if x['unit']=='litres' else x['unit'], axis=1)




In [603]:
clean_df

,type,Data_value,measure,unit
0,Oranges,4.27,1.000,kg
1,Bananas,3.00,1.000,kg
2,Apples,4.62,1.000,kg
3,Kiwifruit,7.78,1.000,kg
4,Sultanas,2.23,0.375,kg
...,...,...,...,...
150,Mayonnaise,2.92,0.380,litre
151,Dried mixed herbs,2.60,0.015,kg
152,Peanuts,3.49,0.250,kg
153,Ham,13.88,1.000,kg


In [647]:
food_types = pd.DataFrame(clean_df['type'].unique(), columns=['food_type'])

In [648]:
food_types

,food_type
0,Oranges
1,Bananas
2,Apples
3,Kiwifruit
4,Sultanas
...,...
135,Mayonnaise
136,Dried mixed herbs
137,Peanuts
138,Ham


In [649]:
food_types['protein'] = None
food_types['fat'] = None
food_types['carb'] = None
food_types['sugar'] = None
food_types['sodium'] = None
food_types['cholesterol'] = None
food_types['calcium'] = None
food_types['kcal'] = None
food_types['size'] = None
food_types['unit'] = None
food_types['category'] = None

### Applying the API to the clean nz table

In [650]:
#applying the nutrients_super
for index, row in food_types.iterrows():
    food_nutrients = nutrients_super(row['food_type'])
    food_types.loc[index, 'protein'] = food_nutrients['protein']
    food_types.loc[index, 'fat'] = food_nutrients['fat']
    food_types.loc[index, 'carb'] = food_nutrients['carb']
    food_types.loc[index, 'sugar'] = food_nutrients['sugar']
    food_types.loc[index, 'cholesterol'] = food_nutrients['cholesterol']
    food_types.loc[index, 'sodium'] = food_nutrients['sodium']
    food_types.loc[index, 'calcium'] = food_nutrients['calcium']
    food_types.loc[index, 'kcal'] = food_nutrients['kcal']
    food_types.loc[index, 'size'] = food_nutrients['size']
    food_types.loc[index, 'unit'] = food_nutrients['unit']
    food_types.loc[index, 'category'] = food_nutrients['category']

missing foundation info for Oranges
foundation
missing foundation info for Bananas
missing branded info for Bananas
switched to branded
missing foundation info for Apples
foundation
missing foundation info for Kiwifruit
foundation
missing foundation info for Sultanas
crashed, forced nutrients_redux
missing foundation info for Peaches  canned
foundation
missing foundation info for Lettuce
foundation
missing foundation info for Broccoli
foundation
missing foundation info for Cabbage
crashed, forced nutrients_redux
missing branded info for Cabbage
missing foundation info for Tomatoes
switched to branded
missing foundation info for Carrots
foundation
missing foundation info for Mushrooms
switched to branded
missing foundation info for Potatoes
crashed, forced nutrients_redux
missing foundation info for Peas  frozen
foundation
missing foundation info for Beef steak  blade
foundation
missing foundation info for Beef steak  porterhouse sirloin
foundation
missing foundation info for Beef  minc

missing branded info for Two minute noodles
missing foundation info for Peanut butter
foundation
missing foundation info for Sweets
foundation
missing foundation info for Cakes and biscuits
crashed, forced nutrients_redux
missing branded info for Cakes and biscuits
missing foundation info for Cookie
foundation
missing foundation info for Takeaway muffins and buns
crashed, forced nutrients_redux
missing branded info for Takeaway muffins and buns
missing foundation info for Pizza
crashed, forced nutrients_redux
missing foundation info for Tea
crashed, forced nutrients_redux
missing foundation info for Vinegar
crashed, forced nutrients_redux
missing branded info for Vinegar
missing foundation info for Wheatmeal bread
foundation
missing foundation info for Wholegrain bread
foundation
missing foundation info for Fresh herbs
foundation
missing foundation info for Olives
switched to branded
missing foundation info for Margarine table spread
missing branded info for Margarine table spread
swit

In [771]:
#correcting the sugar column, this should be ran once because the api is already corrected
for index, row in food_types.iterrows():
    food_nutrients = nutrients_super(row['food_type'])
    food_types.loc[index, 'sugar'] = food_nutrients['sugar']
   

missing foundation info for Oranges
foundation
missing foundation info for Bananas
missing branded info for Bananas
switched to branded
missing foundation info for Apples
foundation
missing foundation info for Kiwifruit
foundation
missing foundation info for Sultanas
crashed, forced nutrients_redux
missing foundation info for Peaches  canned
foundation
missing foundation info for Lettuce
foundation
missing foundation info for Broccoli
foundation
missing foundation info for Cabbage
crashed, forced nutrients_redux
missing branded info for Cabbage
missing foundation info for Tomatoes
foundation
missing foundation info for Carrots
foundation
missing foundation info for Mushrooms
foundation
missing foundation info for Potatoes
crashed, forced nutrients_redux
missing foundation info for Peas  frozen
foundation
missing foundation info for Beef steak  blade
foundation
missing foundation info for Beef steak  porterhouse sirloin
foundation
missing foundation info for Beef  mince
foundation
missi

missing foundation info for Peanut butter
foundation
missing foundation info for Sweets
foundation
missing foundation info for Cakes and biscuits
crashed, forced nutrients_redux
missing branded info for Cakes and biscuits
missing foundation info for Cookie
foundation
missing foundation info for Takeaway muffins and buns
crashed, forced nutrients_redux
missing branded info for Takeaway muffins and buns
missing foundation info for Pizza
crashed, forced nutrients_redux
missing foundation info for Tea
crashed, forced nutrients_redux
missing foundation info for Vinegar
crashed, forced nutrients_redux
missing branded info for Vinegar
missing foundation info for Wheatmeal bread
foundation
missing foundation info for Wholegrain bread
foundation
missing foundation info for Fresh herbs
foundation
missing foundation info for Olives
switched to branded
missing foundation info for Margarine table spread
missing branded info for Margarine table spread
switched to branded
missing foundation info for 

In [651]:
food_types['unit'].value_counts()

g     137
ml      3
Name: unit, dtype: int64

In [652]:
#nutrients_super('Sultanas')

In [653]:
food_types['protein'].value_counts()/len(food_types)

0.00     0.100000
9.43     0.035714
16.70    0.028571
3.84     0.028571
1.41     0.021429
           ...   
23.20    0.007143
8.00     0.007143
7.02     0.007143
2.96     0.007143
19.60    0.007143
Name: protein, Length: 92, dtype: float64

In [654]:
food_types['unit'].value_counts()/len(food_types)

g     0.978571
ml    0.021429
Name: unit, dtype: float64

In [657]:
food_types['category'].nunique()

62

In [774]:
#looking for rows in cholesterol with not found value
print(len(food_types.loc[food_types['sugar']== 'not found'].sort_values(by='food_type')))
food_types.loc[food_types['sugar']== 'not found'].sort_values(by='food_type')

0


,food_type,protein,fat,carb,sugar,sodium,cholesterol,calcium,kcal,size,unit,category


In [750]:
#replacing the not found values with 0
food_types['cholesterol'].replace('not found', 0, inplace=True)

In [754]:
#food_types

In [778]:
nutrients_redux_2('coconut')

missing foundation info for coconut


{'protein': 0.0,
 'fat': 99.1,
 'carb': 0.84,
 'sugar': 90.5,
 'sodium': 0.0,
 'cholesterol': 'not found',
 'calcium': 1.0,
 'kcal': 833,
 'size': 100,
 'unit': 'g',
 'category': 'Fats and Oils'}

In [770]:
nutrients_redux('Bananas')

missing branded info for Bananas


{'protein': 3.89,
 'fat': 1.81,
 'carb': 88.3,
 'sugar': 2.0,
 'sodium': 3.0,
 'cholesterol': 0.0,
 'calcium': 22.0,
 'kcal': 346,
 'size': 100,
 'unit': 'g',
 'category': 'Fruits and Fruit Juices'}

In [763]:
nutrients('Yoghurt flavoured')

Yoghurt flavoured has 177 MG of Calcium, Ca
Yoghurt flavoured has 0.0 MG of Iron, Fe
Yoghurt flavoured has 177 IU of Vitamin A, IU
Yoghurt flavoured has 0.0 MG of Vitamin C, total ascorbic acid
Yoghurt flavoured has 5.31 G of Protein
Yoghurt flavoured has 5.31 G of Total lipid (fat)
Yoghurt flavoured has 15.0 G of Carbohydrate, by difference
Yoghurt flavoured has 124 KCAL of Energy
Yoghurt flavoured has 13.3 G of Sugars, total including NLEA
Yoghurt flavoured has 49.0 MG of Sodium, Na
Yoghurt flavoured has 13.0 MG of Cholesterol
Yoghurt flavoured has 0.0 G of Fatty acids, total trans
Yoghurt flavoured has 2.65 G of Fatty acids, total saturated
Yoghurt flavoured has 0.9 G of Fiber, total dietary


In [637]:
def nut(index, nutrient, value):
    
    food_types.loc[index][nutrient] = value
        

In [741]:

nut(41, 'cholesterol', 0)

In [742]:
food_types.loc[41]

food_type                   Tomato sauce  canned
protein                                     0.84
fat                                          0.5
carb                                        3.32
sugar                                       2.99
sodium                                       125
cholesterol                                    0
calcium                                     30.0
kcal                                        76.0
size                                         100
unit                                           g
category       Vegetables and Vegetable Products
Name: 41, dtype: object

In [773]:
#cheking for missing values
for nutrient in food_types:
    missing = len(food_types[nutrient].loc[food_types[nutrient]== 'not found'])
    
    print(nutrient, 'is missing', missing/len(food_types))

food_type is missing 0.0
protein is missing 0.0
fat is missing 0.007142857142857143
carb is missing 0.007142857142857143
sugar is missing 0.0
sodium is missing 0.014285714285714285
cholesterol is missing 0.0
calcium is missing 0.05
kcal is missing 0.03571428571428571
size is missing 0.0
unit is missing 0.0
category is missing 0.0


In [123]:
for nutrient in food_types:
    missing = len(food_types[nutrient].loc[food_types[nutrient]== 'not found'])
    
    print(nutrient, 'is missing', missing/len(food_types), 'percent')

food_type is missing 0.0 percent
protein is missing 0.02127659574468085 percent
fat is missing 0.028368794326241134 percent
carb is missing 0.028368794326241134 percent
sugar is missing 0.9574468085106383 percent
sodium is missing 0.028368794326241134 percent
cholesterol is missing 0.05673758865248227 percent
calcium is missing 0.0851063829787234 percent
kcal is missing 0.02127659574468085 percent
size is missing 0.5815602836879432 percent
unit is missing 0.5815602836879432 percent


In [779]:
clean_df.shape

(154, 4)

In [90]:
#basic manual api call
API_KEY = 'zYKJIFsOxODivrK9dGg8Y2UUWxkr2j2HMNYXLLwf'

product='fish'

params = {
    'api_key': API_KEY,
    'query': product,
    'pageSize': 5
}

**Converting the units of nutrients to kg or litre**

In [786]:
food_types['unit'].unique()

array(['g', 'ml'], dtype=object)

In [ ]:
nut_convert = food_types

## Joining the food types table with the nz clean

First we will replace the not found values for size and units with 100 g, that's because the documentation of the API specifies the standard for those features were 100 g.

In [780]:
food_types

,food_type,protein,fat,carb,sugar,sodium,cholesterol,calcium,kcal,size,unit,category
0,Oranges,0.91,0.15,11.8,10.6,9.0,0.0,43.0,196,100,g,Fruits and Fruit Juices
1,Bananas,3.89,1.81,88.3,2.0,3.0,0.0,22.0,346,100,g,Fruits and Fruit Juices
2,Apples,0.148,0.162,15.7,13.3,1.01,0.0,5.98,not found,100,g,Fruits and Fruit Juices
3,Kiwifruit,1.06,0.44,14.0,241,5.0,0.0,35.0,241,100,g,Fruits and Fruit Juices
4,Sultanas,2.5,0.0,77.5,0.0,0.0,0.0,50.0,325,40.0,g,Wholesome Snacks
...,...,...,...,...,...,...,...,...,...,...,...,...
135,Mayonnaise,0.0,75.0,0.0,12.5,667,83.0,0.0,750,12.0,g,Salad Dressing & Mayonnaise
136,Dried mixed herbs,79.9,0.65,6.02,12.8,1250.0,17.0,104,376,100,g,Dairy and Egg Products
137,Peanuts,25.0,53.6,17.9,14.3,393,0.0,0.0,607,28.0,g,"Popcorn, Peanuts, Seeds & Related Snacks"
138,Ham,19.6,3.68,2.36,507,1030.0,45.0,6.0,507,100,g,Pork Products


**Joining the tables**

In [138]:
food_types.head()

,food_type,protein,fat,carb,sugar,sodium,cholesterol,calcium,kcal,size,unit
0,Oranges,0.65,0.15,12.3,not found,1.0,0.0,45.0,49.0,154.0,g
1,Bananas,3.89,1.81,88.3,not found,3.0,0.0,22.0,346,100.0,g
2,Apples,0.0,0.0,14.3,not found,0.0,0.0,0.0,52.0,154.0,g
3,Kiwifruit,0.68,0.68,13.5,not found,0.0,0.0,14.0,61.0,148.0,g
4,Sultanas,2.5,0.0,77.5,not found,0.0,0.0,50.0,325,40.0,g


In [823]:
merged_df = pd.merge(left=clean_df, right=food_types, left_on='type', right_on='food_type')

In [824]:
merged_df.head()

,type,Data_value,measure,unit_x,food_type,protein,fat,carb,sugar,sodium,cholesterol,calcium,kcal,size,unit_y,category
0,Oranges,4.27,1.000,kg,Oranges,0.91,0.15,11.8,10.6,9.0,0.0,43.0,196,100,g,Fruits and Fruit Juices
1,Bananas,3.00,1.000,kg,Bananas,3.89,1.81,88.3,2.0,3.0,0.0,22.0,346,100,g,Fruits and Fruit Juices
2,Apples,4.62,1.000,kg,Apples,0.148,0.162,15.7,13.3,1.01,0.0,5.98,not found,100,g,Fruits and Fruit Juices
3,Kiwifruit,7.78,1.000,kg,Kiwifruit,1.06,0.44,14.0,241,5.0,0.0,35.0,241,100,g,Fruits and Fruit Juices
4,Sultanas,2.23,0.375,kg,Sultanas,2.5,0.0,77.5,0.0,0.0,0.0,50.0,325,40.0,g,Wholesome Snacks


In [825]:
#dropping uneccesary columns such as UNITS, sugar and food_type

In [826]:
merged_df.drop(columns=['food_type'], inplace=True)

In [832]:
merged_df = merged_df.replace('not found', np.nan)

In [833]:
merged_df.loc[merged_df['fat']=='not found']

,type,Data_value,measure,unit_x,protein,fat,carb,sugar,sodium,cholesterol,calcium,kcal,size,unit_y,category


In [834]:
merged_df['protein'] = pd.to_numeric(merged_df['protein'])
merged_df['fat'] = pd.to_numeric(merged_df['fat'])
merged_df['carb'] = pd.to_numeric(merged_df['carb'])
merged_df['sugar'] = pd.to_numeric(merged_df['sugar'])
merged_df['sodium'] = pd.to_numeric(merged_df['sodium'])
merged_df['cholesterol'] = pd.to_numeric(merged_df['cholesterol'])
merged_df['calcium'] = pd.to_numeric(merged_df['calcium'])

In [835]:
merged_df['protein'] = merged_df['protein'] / merged_df['size'] * (merged_df['measure']*1000)
merged_df['fat'] = merged_df['fat'] / merged_df['size'] * (merged_df['measure']*1000)
merged_df['carb'] = merged_df['carb'] / merged_df['size'] * (merged_df['measure']*1000)
merged_df['sugar'] = merged_df['sugar'] / merged_df['size'] * (merged_df['measure']*1000)
merged_df['sodium'] = merged_df['sodium'] / merged_df['size'] * (merged_df['measure']*1000)
merged_df['cholesterol'] = merged_df['cholesterol'] / merged_df['size'] * (merged_df['measure']*1000)
merged_df['calcium'] = merged_df['calcium'] / merged_df['size'] * (merged_df['measure']*1000)
merged_df['kcal'] = merged_df['kcal'] / merged_df['size'] * (merged_df['measure']*1000)

In [836]:
merged_df.head()

,type,Data_value,measure,unit_x,protein,fat,carb,sugar,sodium,cholesterol,calcium,kcal,size,unit_y,category
0,Oranges,4.27,1.000,kg,9.1000,1.50,118.0000,106.0,90.0,0.0,430.00,196.0,100.0,g,Fruits and Fruit Juices
1,Bananas,3.00,1.000,kg,38.9000,18.10,883.0000,20.0,30.0,0.0,220.00,346.0,100.0,g,Fruits and Fruit Juices
2,Apples,4.62,1.000,kg,1.4800,1.62,157.0000,133.0,10.1,0.0,59.80,NaN,100.0,g,Fruits and Fruit Juices
3,Kiwifruit,7.78,1.000,kg,10.6000,4.40,140.0000,2410.0,50.0,0.0,350.00,241.0,100.0,g,Fruits and Fruit Juices
4,Sultanas,2.23,0.375,kg,23.4375,0.00,726.5625,0.0,0.0,0.0,468.75,325.0,40.0,g,Wholesome Snacks


## Declaring  X and y, and replacing ''not found'' with np.nan

In [837]:
y = merged_df['Data_value']
X= merged_df[['protein','fat','carb','sugar','sodium','cholesterol','calcium','kcal','category']]

### Using the pipeline to preprocess X

In [838]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()

k_imputer = KNNImputer(n_neighbors=4)

In [839]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


num_trans = Pipeline([
    ('imputer', KNNImputer(n_neighbors=4)),
    ('sclaer', MinMaxScaler())
])

cat_trans = OneHotEncoder(handle_unknown='ignore', sparse=False)

preprocessor = ColumnTransformer([
    ('num_trans', num_trans,['protein','fat','carb','sugar','sodium','cholesterol','calcium','kcal']),
    ('cat_trans', cat_trans, ['category'])
])

#['protein','fat','carb','sodium','calcium','kcal','2019']

In [840]:
X_t =  preprocessor.fit_transform(X)
X_t

array([[1.46443515e-02, 6.31578947e-04, 2.36947791e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [6.26005793e-02, 7.62105263e-03, 1.77309237e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.38171870e-03, 6.82105263e-04, 3.15261044e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [3.59211918e-01, 2.01503759e-01, 3.20926563e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [3.15416801e-01, 1.54947368e-02, 4.73895582e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 3.53684211e-05, 5.43694779e-03, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

### spliting x and y

In [841]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [849]:
X_train, X_test, y_train, y_test = train_test_split(X_t, y, test_size=0.3)

## Running the models

### Basic linear model

In [850]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate

In [851]:
base_model = LinearRegression()

cv_results = cross_validate(base_model, X_train, y_train, cv=5, scoring = ['max_error','r2','neg_mean_squared_error','neg_mean_absolute_error'])
cv_results

{'fit_time': array([0.00371528, 0.00347209, 0.0018754 , 0.0019567 , 0.00221753]),
 'score_time': array([0.00349474, 0.00167656, 0.00132442, 0.00181365, 0.00108814]),
 'test_max_error': array([-3.03021877e+13, -1.53090119e+13, -3.56686843e+13, -4.65225475e+13,
        -1.19031039e+14]),
 'test_r2': array([-2.72637290e+24, -1.38916160e+25, -1.19967638e+25, -1.51364045e+25,
        -5.00688577e+25]),
 'test_neg_mean_squared_error': array([-5.78292082e+25, -5.04857861e+25, -2.48757995e+26, -5.48517851e+26,
        -1.71217523e+27]),
 'test_neg_mean_absolute_error': array([-3.08786483e+12, -3.38171628e+12, -8.90923019e+12, -1.23352121e+13,
        -2.10783743e+13])}

In [852]:
base_results = {
    'test_max_error': cv_results['test_max_error'].mean(),
    'test_r2': cv_results['test_r2'].mean(),
    'test_neg_mean_squared_error': cv_results['test_neg_mean_squared_error'].mean(),
    'test_neg_mean_absolute_error': cv_results['test_neg_mean_absolute_error'].mean()
}
base_results

{'test_max_error': -49366693984081.05,
 'test_r2': -1.8764002967198844e+25,
 'test_neg_mean_squared_error': -5.235532143291406e+26,
 'test_neg_mean_absolute_error': -9758479542141.215}

In [853]:
base_model.fit(X_train, y_train)
base_model.score(X_test, y_test)

-1.5401853940918517e+26

In [854]:
scores = cross_val_score(base_model, X_train, y_train, cv=10)
scores.mean()

-8.678011769311465e+25

### Exploring responses (do not use, it is part of a try and error)

In [872]:
try:
    product = 'banana'
    params = {
        'api_key': API_KEY,
        'query': product,
        'pageSize': 1,
        'numberOfResultsPerPage':1
        #'dataType': "Foundation"
    }
    response = requests.get('https://api.nal.usda.gov/fdc/v1/foods/search', params=params).json()
    
    print(response['foods'][0]['servingSize'], response['foods'][0]['servingSizeUnit'])
except:
    print('not found')

32.0 g


In [873]:
response['foods']

[{'fdcId': 2012128,
  'description': 'BANANA',
  'lowercaseDescription': 'banana',
  'dataType': 'Branded',
  'gtinUpc': '016459200441',
  'publishedDate': '2021-10-28',
  'brandOwner': 'Wonder Natural Foods Corp',
  'brandName': "BETTER'N PEANUT BUTTER",
  'ingredients': 'PEANUTS (AS DEFATTED PEANUT FLOUR, PEANUT BUTTER AND NATURAL PEANUT OILS), PURE WATER, TAPIOCA SYRUP, RICE SYRUP, VEGETABLE GLYCERINE, CANE SUGAR, BANANA, NATURAL COLORS AND FLAVORS, SALT, CALCIUM CARBONATE, LECITHIN, TOCOPHEROL (VITAMIN E), SODIUM ASCORBATE (VITAMIN C).',
  'marketCountry': 'United States',
  'foodCategory': 'Nut & Seed Butters',
  'modifiedDate': '2018-06-29',
  'dataSource': 'LI',
  'packageWeight': '16 oz/1 lbs/454 g',
  'servingSizeUnit': 'g',
  'servingSize': 32.0,
  'allHighlightFields': '<b>Ingredients</b>:  SYRUP, RICE SYRUP, VEGETABLE GLYCERINE, CANE SUGAR, <em>BANANA</em>, NATURAL COLORS AND FLAVORS, SALT, CALCIUM',
  'score': 1033.3203,
  'foodNutrients': [{'nutrientId': 1003,
    'nutrie

In [874]:
response

{'totalHits': 4307,
 'currentPage': 1,
 'totalPages': 4307,
 'pageList': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
 'foodSearchCriteria': {'query': 'banana',
  'generalSearchInput': 'banana',
  'pageNumber': 1,
  'numberOfResultsPerPage': 1,
  'pageSize': 1,
  'requireAllWords': False},
 'foods': [{'fdcId': 2012128,
   'description': 'BANANA',
   'lowercaseDescription': 'banana',
   'dataType': 'Branded',
   'gtinUpc': '016459200441',
   'publishedDate': '2021-10-28',
   'brandOwner': 'Wonder Natural Foods Corp',
   'brandName': "BETTER'N PEANUT BUTTER",
   'ingredients': 'PEANUTS (AS DEFATTED PEANUT FLOUR, PEANUT BUTTER AND NATURAL PEANUT OILS), PURE WATER, TAPIOCA SYRUP, RICE SYRUP, VEGETABLE GLYCERINE, CANE SUGAR, BANANA, NATURAL COLORS AND FLAVORS, SALT, CALCIUM CARBONATE, LECITHIN, TOCOPHEROL (VITAMIN E), SODIUM ASCORBATE (VITAMIN C).',
   'marketCountry': 'United States',
   'foodCategory': 'Nut & Seed Butters',
   'modifiedDate': '2018-06-29',
   'dataSource': 'LI',
   'packageWeight': '

In [238]:
nutrients('apple')

apple has 0.0 MG of Calcium, Ca
apple has 0.23 MG of Iron, Fe
apple has 0.0 MG of Sodium, Na
apple has 65.0 IU of Vitamin A, IU
apple has 3.1 MG of Vitamin C, total ascorbic acid
apple has 0.0 MG of Cholesterol
apple has 0.0 G of Fatty acids, total saturated
apple has 0.0 G of Protein
apple has 14.3 G of Carbohydrate, by difference
apple has 52.0 KCAL of Energy
apple has 10.4 G of Sugars, total including NLEA
apple has 3.2 G of Fiber, total dietary
apple has 110 MG of Potassium, K
apple has 0.0 G of Fatty acids, total trans
apple has 0.65 G of Total lipid (fat)


In [297]:
nutrients_super('Beef steak  porterhouse sirloin')

missing foundation info for Beef steak  porterhouse sirloin
foundation


{'protein': 22.7,
 'fat': 5.32,
 'carb': 0.0,
 'sugar': 'not found',
 'sodium': 43.0,
 'cholesterol': 57.0,
 'calcium': 19.0,
 'kcal': 606,
 'size': 100,
 'unit': 'g',
 'category': 'Beef Products'}

In [227]:
nutrients_redux_2('orange')

missing foundation info for orange


{'protein': 0.91,
 'fat': 0.15,
 'carb': 11.8,
 'sugar': 8.57,
 'sodium': 9.0,
 'cholesterol': 'not found',
 'calcium': 43.0,
 'kcal': 196,
 'size': 'not found',
 'unit': 'not found',
 'product': 'Fruits and Fruit Juices'}

In [229]:
nutrients_super('orange')

missing foundation info for orange
switched to branded


{'protein': 4.76,
 'fat': 0.0,
 'carb': 90.5,
 'sugar': 'not found',
 'sodium': 381,
 'cholesterol': 'not found',
 'calcium': 'not found',
 'kcal': 381,
 'size': 21.0,
 'unit': 'g',
 'product': 'Gelatin, Gels, Pectins & Desserts'}

In [233]:
nutrients_redux('orange')

{'protein': 4.76,
 'fat': 0.0,
 'carb': 90.5,
 'sugar': 'not found',
 'sodium': 381,
 'cholesterol': 'not found',
 'calcium': 'not found',
 'kcal': 381,
 'size': 21.0,
 'unit': 'g',
 'category': 'Gelatin, Gels, Pectins & Desserts'}